In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split,PredefinedSplit,GridSearchCV
import pickle

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pulp import *
import pandas as pd
import os, glob
import seaborn as sns
from scipy.stats import kruskal
import scikit_posthocs as sp
from scipy.stats import mannwhitneyu
from dotenv import load_dotenv
from google.cloud import bigquery
project_id = os.getenv('GOOGLE_CLOUD_PROJECT')
client = bigquery.Client(project=project_id)
%load_ext google.cloud.bigquery



/Users/sandychen/Desktop/Healthrex_workspace/scripts/Blood_Culture_Stewardship/blood_culture_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/sandychen/Desktop/Healthrex_workspace/scripts/Blood_Culture_Stewardship/blood_culture_env/lib/python3.9/site-packages/google/cloud/bigquery/__init__.py:237: FutureWarning: %load_ext google.cloud.bigquery is deprecated. Install bigquery-magics package and use `%load_ext bigquery_magics`, instead.
  warnings.warn(


# Util funcitons block

In [19]:
def query_bigquery(query):
    return client.query(query).to_dataframe()


In [20]:
train_query = """
select * from `som-nero-phi-jonc101.blood_culture_stewardship.cohort` where order_year>=2015 and order_year<2022
"""
val_query = """
select * from `som-nero-phi-jonc101.blood_culture_stewardship.cohort` where order_year>=2022 and order_year<2023
"""
test_query = """
select * from `som-nero-phi-jonc101.blood_culture_stewardship.cohort` where order_year>=2023 
"""
Train_set_df = query_bigquery(train_query)
Val_set_df = query_bigquery(val_query)
Test_set_df = query_bigquery(test_query)


In [22]:
def process_datetime_features(df, datetime_columns, time_diff_pairs):
    """
    Process datetime columns and calculate time differences for a dataframe.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        The dataframe to process
    datetime_columns : list
        List of column names to convert to datetime
    time_diff_pairs : list of tuples
        List of tuples containing (col1, col2, new_col_name) for time difference calculations
        where new_col_name will contain hours between col1 and col2
    
    Returns:
    --------
    pandas.DataFrame
        The processed dataframe with datetime conversions and time differences
    """
    # Convert specified columns to datetime
    for col in datetime_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col])
    
    # Calculate time differences in hours
    for col1, col2, new_col_name in time_diff_pairs:
        if col1 in df.columns and col2 in df.columns:
            df[new_col_name] = (df[col1] - df[col2]).dt.total_seconds() / 3600
    
    return df


In [23]:
# Define datetime columns and time difference calculations
datetime_columns = ['ed_arrival_datetime', 'blood_culture_order_datetime', 'earliest_iv_antibiotic_datetime']
time_diff_pairs = [
    ('blood_culture_order_datetime', 'ed_arrival_datetime', 'hours_between_ed_cult'),
    ('blood_culture_order_datetime', 'earliest_iv_antibiotic_datetime', 'hours_between_cult_abx')
]

# Process all dataframes using the function
Train_set_df = process_datetime_features(Train_set_df, datetime_columns, time_diff_pairs)
Val_set_df = process_datetime_features(Val_set_df, datetime_columns, time_diff_pairs)
Test_set_df = process_datetime_features(Test_set_df, datetime_columns, time_diff_pairs)


In [24]:
def any_abx (row):
    if row['vanc']==1 or row['zosyn']==1 or row['other_ABX']==1:
        return 1
    else:
        return 0

In [25]:
Train_set_df['any_abx'] = Train_set_df.apply(any_abx, axis=1)
Val_set_df['any_abx'] = Val_set_df.apply(any_abx, axis=1)
Test_set_df['any_abx'] = Test_set_df.apply(any_abx, axis=1)

In [30]:
"min_alc" in Train_set_df.columns

False

In [ ]:
Identifiers=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded']
# "blood_culture_order_datetime", "ed_arrival_datetime"]
Labels=['positive_blood_culture','positive_blood_culture_in_week']
# keep vitals, labs(remove min, )
keep_vitals = ['min_heartrate','max_heartrate', 'avg_heartrate', 'median_heartrate',
       'min_resprate', 'max_resprate', 'avg_resprate', 'median_resprate',
       'min_temp', 'max_temp', 'avg_temp', 'median_temp', 'min_sysbp',
       'max_sysbp', 'avg_sysbp', 'median_sysbp', 'min_diasbp',
       'max_diasbp', 'avg_diasbp', 'median_diasbp']

# Labs=['min_heartrate','max_heartrate', 'avg_heartrate', 'median_heartrate',
#        'min_resprate', 'max_resprate', 'avg_resprate', 'median_resprate',
#        'min_temp', 'max_temp', 'avg_temp', 'median_temp', 'min_sysbp',
#        'max_sysbp', 'avg_sysbp', 'median_sysbp', 'min_diasbp',
#        'max_diasbp', 'avg_diasbp', 'median_diasbp', 
#        'min_wbc', 'max_wbc',
# add Absolute neutrophil count (ANC), Absolute lymphocyte count (ALC),C reactive protein (CRP)
# 
labs = [
       'avg_wbc', 'median_wbc', 'min_neutrophils', 'max_neutrophils',
       'avg_neutrophils', 'median_neutrophils', 'min_lymphocytes',
       'max_lymphocytes', 'avg_lymphocytes', 'median_lymphocytes',
       'min_hgb', 'max_hgb', 'avg_hgb', 'median_hgb', 'min_plt',
       'max_plt', 'avg_plt', 'median_plt', 'min_na', 'max_na', 'avg_na',
       'median_na', 'min_hco3', 'max_hco3', 'avg_hco3', 'median_hco3',
       'min_bun', 'max_bun', 'avg_bun', 'median_bun', 'min_cr', 'max_cr',
       'avg_cr', 'median_cr', 'min_lactate', 'max_lactate', 'avg_lactate',
       'median_lactate', 'min_procalcitonin', 'max_procalcitonin',
       'avg_procalcitonin', 'median_procalcitonin']
Demos=[ 'gender','age']
# ABX=['vanc', 'zosyn', 'vanc_zosyn', 'other_ABX']
ABX = ["any_abx"]
Time_Varient_features=['hours_between_ed_cult'] 
# 'hours_between_cult_abx']
Diagnosis= ['bacteremia', 'septic_shock', 'infective_endocarditis',
       'septic_thrombophlebitis', 'vascular_graft_infection', 'CRBSI',
       'infectious_discitis', 'epidural_abscess', 'septic_arthritis',
       'meningitis', 'meningitis_bacteria', 'cholangitis',
       'bacterial_cholangitis', 'pyelonephritis',
       'acute_bacterial_pyelonephritis', 'severe_pneumonia',
       'acute_hematogenous_osteomyelitis', 'asplenia',
       'immunocompromised_state', 'severe_cellulitis', 'cystitis',
       'prostatitis', 'CAP', 'diabetic_foot_infection', 'colitis',
       'aspiration_pneumonia', 'uncomplicated_cholecystitis',
       'uncomplicated_diverticulitis', 'Uncomplicated_pancreatitis']
# remove diagnosis

Feature_set=Identifiers+Labels+Labs+Demos+ABX+Diagnosis+Time_Varient_features #(select Features based on experiment)
Train_set_df=Train_set_df[Feature_set]
Test_set_df=Test_set_df[Feature_set]
Val_set_df=Val_set_df[Feature_set]
                       

In [ ]:
Test_set_df["any_abx"].

0        0
1        1
2        0
3        0
4        0
        ..
25363    0
25364    1
25365    1
25366    1
25367    1
Name: any_abx, Length: 25368, dtype: int64

In [24]:
len(Feature_set) # exlucde the identifiers

106

In [58]:
print(f"the size of train set is {Train_set_df.shape[0]}")
print(f"the size of val set is {Val_set_df.shape[0]}")
print(f"the size of test set is {Test_set_df.shape[0]}")

the size of train set is 90708
the size of val set is 19407
the size of test set is 25368


In [25]:
# Train_set_df.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'],inplace=True)
# Val_set_df.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'],inplace=True)
# Test_set_df.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'],inplace=True)
# # shouldn't we report this number after deduplication?
# # why do we need to deduplicate?
Train_set_df.drop_duplicates(subset = Identifiers, inplace= True)
Val_set_df.drop_duplicates(subset = Identifiers, inplace= True)
Test_set_df.drop_duplicates(subset = Identifiers, inplace= True)


/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_95895/4188508575.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Val_set_df.drop_duplicates(subset = Identifiers, inplace= True)
/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_95895/4188508575.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test_set_df.drop_duplicates(subset = Identifiers, inplace= True)


In [26]:
print(f"the size of train set is {Train_set_df.shape[0]}")
print(f"the size of val set is {Val_set_df.shape[0]}")
print(f"the size of test set is {Test_set_df.shape[0]}")

the size of train set is 90708
the size of val set is 19407
the size of test set is 25368


In [27]:
# why do we need the datapoint?
Train_set_df['datapoint'] = Train_set_df.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded']).ngroup() + 1
Test_set_df['datapoint'] = Train_set_df.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded']).ngroup() + 1
Val_set_df['datapoint'] = Val_set_df.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded']).ngroup() + 1


/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_95895/3045874452.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test_set_df['datapoint'] = Train_set_df.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded']).ngroup() + 1
/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_95895/3045874452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Val_set_df['datapoint'] = Val_set_df.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded']).ngroup() + 1


In [28]:
Train_set_df['gender'] = Train_set_df['gender'].apply(lambda x: 1 if x == 'Male' else (0 if x == 'Female' else None))
Test_set_df['gender'] = Test_set_df['gender'].apply(lambda x: 1 if x == 'Male' else (0 if x == 'Female' else None))
Val_set_df['gender'] = Val_set_df['gender'].apply(lambda x: 1 if x == 'Male' else (0 if x == 'Female' else None))


/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_95895/909712918.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test_set_df['gender'] = Test_set_df['gender'].apply(lambda x: 1 if x == 'Male' else (0 if x == 'Female' else None))
/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_95895/909712918.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Val_set_df['gender'] = Val_set_df['gender'].apply(lambda x: 1 if x == 'Male' else (0 if x == 'Female' else None))


In [29]:
Train_set_df.shape[0]+Test_set_df.shape[0]+Val_set_df.shape[0]

135483

# Logistic Regression

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
import statsmodels.api as sm
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
# from fancyimpute import IterativeImputer as FancyIterativeImputer

In [31]:
Train_set_df['Label']= (Train_set_df['positive_blood_culture']|Train_set_df['positive_blood_culture_in_week'])
X_train = Train_set_df.drop(columns=['positive_blood_culture', 'positive_blood_culture_in_week','anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','Label'])
# 'blood_culture_order_datetime',
#  'ed_arrival_datetime' ])
y_train = Train_set_df['Label']

# Prepare the test data
Test_set_df['Label']= (Test_set_df['positive_blood_culture']|Test_set_df['positive_blood_culture_in_week'])
X_test = Test_set_df.drop(columns=['positive_blood_culture', 'positive_blood_culture_in_week','anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded', 'Label'])
# "blood_culture_order_datetime", "ed_arrival_datetime"]) # this line was problematic. I fixed by adding "Lable"
y_test = Test_set_df['Label']


Val_set_df['Label']= (Val_set_df['positive_blood_culture']|Val_set_df['positive_blood_culture_in_week'])
X_val = Val_set_df.drop(columns=['positive_blood_culture', 'positive_blood_culture_in_week','anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded', 'Label'] )
# "blood_culture_order_datetime", "ed_arrival_datetime"])
y_val = Val_set_df['Label']

/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_95895/2775927307.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test_set_df['Label']= (Test_set_df['positive_blood_culture']|Test_set_df['positive_blood_culture_in_week'])
/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_95895/2775927307.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Val_set_df['Label']= (Val_set_df['positive_blood_culture']|Val_set_df['positive_blood_culture_in_week'])


In [ ]:
X_train.columns

Index(['min_heartrate', 'max_heartrate', 'avg_heartrate', 'median_heartrate',
       'min_resprate', 'max_resprate', 'avg_resprate', 'median_resprate',
       'min_temp', 'max_temp',
       ...
       'CAP', 'diabetic_foot_infection', 'colitis', 'aspiration_pneumonia',
       'uncomplicated_cholecystitis', 'uncomplicated_diverticulitis',
       'Uncomplicated_pancreatitis', 'hours_between_ed_cult',
       'hours_between_cult_abx', 'datapoint'],
      dtype='object', length=102)

In [66]:
y_train.mean()

np.float64(0.07103011862239274)

# SKip Fancy_Imputer for now as the manuscript uses median imputation

## Fancy_Imputer


In [ ]:
fancy_imputer = FancyIterativeImputer(max_iter=10, random_state=0)
fancy_imputer.fit(X_train)

X_train = pd.DataFrame(fancy_imputer.transform(X_train), columns=X_train.columns)

# Impute the missing values in the test set
X_test = pd.DataFrame(fancy_imputer.transform(X_test), columns=X_train_im.columns)

# Impute the missing values in the validation set
X_val = pd.DataFrame(fancy_imputer.transform(X_val), columns=X_train_im.columns)


## Median Imputer

In [33]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
imputer.fit(X_train)  # Fit on the training data to calculate medians
# Step 2: Impute the missing values in all datasets
X_train = pd.DataFrame(imputer.transform(X_train), columns=X_train.columns)
X_val = pd.DataFrame(imputer.transform(X_val), columns=X_val.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)


## For study with including Fever as Features

In [34]:
print(f"the size of train set is {X_train.shape}")
print(f"the size of val set is {X_val.shape}")
print(f"the size of test set is {X_test.shape}")

the size of train set is (90708, 102)
the size of val set is (19407, 102)
the size of test set is (25368, 102)


In [36]:
# retrain
X_train['Fever'] = X_train['max_temp'].apply(lambda x: 1 if x >= 100.4 else 0)
X_val['Fever'] = X_val['max_temp'].apply(lambda x: 1 if x >= 100.4 else 0)
X_test['Fever'] = X_test['max_temp'].apply(lambda x: 1 if x > 100.4 else 0)


In [37]:
print(f"the size of train set is {X_train.shape}")
print(f"the size of val set is {X_val.shape}")
print(f"the size of test set is {X_test.shape}")

the size of train set is (90708, 103)
the size of val set is (19407, 103)
the size of test set is (25368, 103)


## Scale paramters

In [38]:
from sklearn.preprocessing import StandardScaler
# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data
scaler.fit(X_train)

# Transform the training, test, and validation data
X_train2 = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_test2 = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)
X_val2 = pd.DataFrame(scaler.transform(X_val), columns=X_train.columns)

## Model Training

## Search over paramters

In [39]:
ratio = np.sum(y_train == 0) / np.sum(y_train == 1)

custom_weights = [
    None,                  
    "balanced",                  
    {0: 1, 1: 2},                
    {0: 1, 1: 3},                
    {0: 1, 1: 5},
    {0: 1, 1: ratio} 
]
param_grid = [
    {
        "penalty": ["l1"],
        "solver": ["liblinear", "saga"],
        "C": [0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10],
        "class_weight": custom_weights,
        "max_iter": [1000]
    },
    {
        "penalty": ["l2"],
        "solver": ["liblinear", "saga", "lbfgs", "sag"],
        "C": [0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10],
        "class_weight": custom_weights,
        "max_iter": [1000]
    }
]

In [ ]:
# Concatenate train and val so GridSearchCV 
X_search = np.vstack([X_train2, X_val2])
y_search = np.concatenate([y_train, y_val])

test_fold = np.array([-1] * len(y_train) + [0] * len(y_val))
ps = PredefinedSplit(test_fold)

In [ ]:
base_model = LogisticRegression()

grid = GridSearchCV(
    estimator=base_model,
    param_grid=param_grid,
    scoring="roc_auc",
    cv=ps,               
    refit=True,         
    n_jobs=-1,
    verbose=1
)

grid.fit(X_search, y_search)

In [ ]:
# Save the fitted grid for future use
import pickle
from datetime import datetime

# Create filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
grid_filename = f"fitted_grid_{timestamp}.pkl"

# Save the complete fitted grid
with open(grid_filename, 'wb') as f:
    pickle.dump(grid, f)

print(f"Grid saved as: {grid_filename}")
print(f"Best score: {grid.best_score_:.4f}")
print(f"Best parameters: {grid.best_params_}")
print(f"Total combinations tested: {len(grid.cv_results_['params'])}")


In [40]:
import pickle
grid_filename = "fitted_grid_20250903_145027.pkl"
with open(grid_filename, "rb") as f:
    loaded_grid = pickle.load(f)

# The PredefinedSplit object you passed in is still here
ps = loaded_grid.cv

# Its test_fold attribute tells you which sample went to which fold
test_fold = np.array(ps.test_fold)

n_train = np.sum(test_fold == -1)
n_val   = np.sum(test_fold == 0)

print("Training size:", n_train)
print("Validation size:", n_val)


Training size: 90708
Validation size: 19407


In [ ]:
for i in X_train2.columns:
    print(i)

In [41]:
# TO LOAD THE GRID NEXT TIME (instead of rerunning grid search):
# Uncomment and modify filename when you want to load

# import pickle

# Load the saved grid (replace with your actual filename)
with open("fitted_grid_20250903_145027.pkl", 'rb') as f:
    grid = pickle.load(f)

print(f"Grid loaded successfully!")
print(f"Best score: {grid.best_score_:.4f}")
print(f"Best parameters: {grid.best_params_}")

# Now you can use grid.best_estimator_ directly
best_model = grid.best_estimator_


Grid loaded successfully!
Best score: 0.7919
Best parameters: {'C': 0.001, 'class_weight': {0: 1, 1: np.float64(13.078534844016762)}, 'max_iter': 1000, 'penalty': 'l1', 'solver': 'liblinear'}


## LR model

In [74]:
best_model = grid.best_estimator_
y_pred_prob_test = best_model.predict_proba(X_test2)[:, 1]
auc_test = roc_auc_score(y_test, y_pred_prob_test)

/Users/sandychen/Desktop/Healthrex_workspace/scripts/Blood_Culture_Stewardship/blood_culture_env/lib/python3.9/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [ ]:
# best_params = grid.best_params_.copy()
# model_l2 = LogisticRegression(**best_params)
# model_l2.fit(X_train2, y_train)

# # Predict probabilities for the test set
# y_pred_prob_l2 = model_l2.predict_proba(X_test2)[:, 1]

# # Calculate the AUC
# auc_l2 = roc_auc_score(y_test, y_pred_prob_l2)
# print(f"AUC for the test set with L1 regularization: {auc_l2:.4f}")

In [43]:
#new 
best_params = grid.best_params_.copy()
model_l2 = LogisticRegression(**best_params)
model_l2.fit(X_train2, y_train)

# Predict probabilities for the test set
y_pred_prob_l2 = model_l2.predict_proba(X_test2)[:, 1]

# Calculate the AUC
auc_l2 = roc_auc_score(y_test, y_pred_prob_l2)
print(f"AUC for the test set with L1 regularization: {auc_l2:.4f}")

AUC for the test set with L1 regularization: 0.7967


In [ ]:
# with open("refactor_logistic_regression_model.pkl", 'wb') as f:
#     pickle.dump(model_l2, f)

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import roc_auc_score
# import statsmodels.api as sm
# import numpy as np


# # L2 Regularization (Ridge)
# model_l2 = LogisticRegression(max_iter=2000,class_weight='balanced')#, C=0.01, penalty='l1')#solver='lbfgs'
# model_l2.fit(X_train2, y_train)

# # Predict probabilities for the test set
# y_pred_prob_l2 = model_l2.predict_proba(X_test2)[:, 1]

# # Calculate the AUC
# auc_l2 = roc_auc_score(y_test, y_pred_prob_l2)
# print(f"AUC for the test set with L2 regularization: {auc_l2:.4f}")


# # Calculate the AUC
# yt_pred_prob_l2 = model_l2.predict_proba(X_val2)[:, 1]
# auc_l2 = roc_auc_score(y_val, yt_pred_prob_l2)
# print(f"AUC for the validation set with L2 regularization: {auc_l2:.4f}")

In [44]:
import numpy as np
from sklearn.metrics import roc_curve, confusion_matrix

# Calculate the ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob_l2)

# Find the threshold where sensitivity (TPR) >= 0.9
threshold_index = np.argmax(tpr >= 0.95)
optimal_threshold = thresholds[threshold_index]

# Use the optimal threshold to make binary predictions
y_pred_optimal = (y_pred_prob_l2 >= optimal_threshold).astype(int)

# Calculate the confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_optimal).ravel()

# Calculate sensitivity, specificity, PPV, and NPV
sensitivity = tp / (tp + fn)  # Sensitivity or Recall
specificity = tn / (tn + fp)  # Specificity
ppv = tp / (tp + fp)          # Positive Predictive Value (Precision)
npv = tn / (tn + fn)          # Negative Predictive Value

print(f"Optimal Threshold: {optimal_threshold:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"PPV (Precision): {ppv:.4f}")
print(f"NPV: {npv:.4f}")

Optimal Threshold: 0.2893
Sensitivity: 0.9503
Specificity: 0.3344
PPV (Precision): 0.0752
NPV: 0.9916


In [42]:
# new
import numpy as np
from sklearn.metrics import roc_curve, confusion_matrix

# Calculate the ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob_l2)

# Find the threshold where sensitivity (TPR) >= 0.9
threshold_index = np.argmax(tpr >= 0.95)
optimal_threshold = thresholds[threshold_index]

# Use the optimal threshold to make binary predictions
y_pred_optimal = (y_pred_prob_l2 >= optimal_threshold).astype(int)

# Calculate the confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_optimal).ravel()

# Calculate sensitivity, specificity, PPV, and NPV
sensitivity = tp / (tp + fn)  # Sensitivity or Recall
specificity = tn / (tn + fp)  # Specificity
ppv = tp / (tp + fp)          # Positive Predictive Value (Precision)
npv = tn / (tn + fn)          # Negative Predictive Value

print(f"Optimal Threshold: {optimal_threshold:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"PPV (Precision): {ppv:.4f}")
print(f"NPV: {npv:.4f}")

NameError: name 'y_pred_prob_l2' is not defined

In [ ]:
Test_set_df["y_pred_prob_Cultryx"] = y_pred_prob_l2

In [ ]:
# new
import numpy as np
from sklearn.metrics import roc_curve, confusion_matrix

# Calculate the ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob_l2)

# Find the threshold where sensitivity (TPR) >= 0.9
threshold_index = np.argmax(tpr >= 0.95)
optimal_threshold = thresholds[threshold_index]

# Use the optimal threshold to make binary predictions
y_pred_optimal = (y_pred_prob_l2 >= optimal_threshold).astype(int)

# Calculate the confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_optimal).ravel()

# Calculate sensitivity, specificity, PPV, and NPV
sensitivity = tp / (tp + fn)  # Sensitivity or Recall
specificity = tn / (tn + fp)  # Specificity
ppv = tp / (tp + fp)          # Positive Predictive Value (Precision)
npv = tn / (tn + fn)          # Negative Predictive Value

print(f"Optimal Threshold: {optimal_threshold:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"PPV (Precision): {ppv:.4f}")
print(f"NPV: {npv:.4f}")

In [ ]:
Test_set_df["y_pred_Cultryx"] = y_pred_optimal

In [ ]:
to_save_df = Test_set_df[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded',"Label", 'y_pred_Cultryx', 'y_pred_prob_Cultryx']]

In [ ]:
to_save_df.to_csv("Cultryx_predictions.csv", index=False)

In [ ]:
# save model
import pickle

# Assuming model_l2 is your trained Logistic Regression model
with open('logistic_regression_modelI.pkl', 'wb') as model_file:
    pickle.dump(model_l2, model_file)

In [77]:
# Get the coefficients
coefficients = model_l2.coef_[0]  # Coefficients for the features
intercept = model_l2.intercept_[0]  # Intercept term

# Print the coefficients and intercept
print("Coefficients:", coefficients)
print("Intercept:", intercept)

Coefficients: [ 0.00000000e+00  6.76946719e-02  0.00000000e+00  9.11883449e-02
  0.00000000e+00  0.00000000e+00  1.69859821e-03  0.00000000e+00
  0.00000000e+00  3.51428537e-01  0.00000000e+00  0.00000000e+00
 -6.37092346e-02  0.00000000e+00  0.00000000e+00 -5.46881773e-02
  0.00000000e+00  0.00000000e+00 -4.70226023e-03 -8.16152066e-02
  8.30847983e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  8.68150846e-02  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -9.39016527e-02
  0.00000000e+00 -1.84197797e-01  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -4.47934994e-02  0.00000000e+00 -2.42986139e-02
  0.00000000e+00 -9.05524189e-02  0.00000000e+00  0.00000000e+00
  1.59540921e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  5.65706096e-02  0.00000000e+00  4.34165031e-02  0.00000000e+00
  0.0000000

In [78]:
coefficients = model_l2.coef_[0] 
# Get the feature names from your training data (assuming X_train2 is a DataFrame)
feature_names = X_train2.columns

# Identify non-zero coefficients and get corresponding feature names
non_zero_indices = np.where(coefficients != 0)[0]
non_zero_features = feature_names[non_zero_indices]
non_zero_coefficients = coefficients[non_zero_indices]

# Print the features and their corresponding non-zero coefficients
for feature, coef in zip(non_zero_features, non_zero_coefficients):
    print(f"Feature: {feature}, Coefficient: {coef}")

Feature: max_heartrate, Coefficient: 0.06769467188954216
Feature: median_heartrate, Coefficient: 0.09118834489720277
Feature: avg_resprate, Coefficient: 0.0016985982087139775
Feature: max_temp, Coefficient: 0.35142853663551205
Feature: min_sysbp, Coefficient: -0.06370923459879302
Feature: median_sysbp, Coefficient: -0.05468817727612786
Feature: avg_diasbp, Coefficient: -0.0047022602290733295
Feature: median_diasbp, Coefficient: -0.0816152065758585
Feature: min_wbc, Coefficient: 0.08308479834071922
Feature: avg_neutrophils, Coefficient: 0.08681508462753895
Feature: median_hgb, Coefficient: -0.09390165268955498
Feature: max_plt, Coefficient: -0.18419779694771327
Feature: max_na, Coefficient: -0.044793499403164155
Feature: median_na, Coefficient: -0.0242986138858686
Feature: max_hco3, Coefficient: -0.09055241886663995
Feature: min_bun, Coefficient: 0.15954092094047956
Feature: min_lactate, Coefficient: 0.0565706095633029
Feature: avg_lactate, Coefficient: 0.04341650310866221
Feature: max_